# Protocol: Comparison of Exploration of the Dining Philosophers Problem with Different Number of Processes

Date: 17.09.2021

## Question

Does the number of processes, and by that the state space size, affect the state space exploration performance on the Dining Philosophers Problem?

## Hypothesis

The paper suggests that when the state space size of a model surpasses the hash table capacity, state space exploration slows down significantly, resulting in a much higher number of VTs needed to explore >99% of the state space.

Thus, we expect the exploration to slow down starting with Phils-12 on a $2^{18}$ hash table.

## Setup

- GPU: NVIDIA GeForce RTX 2080 Ti
- Program: `main` branch, commit e160572
- Model: Philosophers model
- CUDA_FLAGS: `-DGRAPPLE_MODEL=PhilosophersStateV2`

## Implementation

To control the number of processes, we have to alter the constants `kProcesses` and `kStateSpaceSize` in `src/models/PhilosophersStateV2.cuh`.

First, we set `kProcesses = 11` and `kStateSpaceSize = 177146`.

We stop the experiment after 10 runs as exhaustive state space coverage is already achieved after the first run.

```
$ time ./build/grapple -s 1736331306 -n 10 -q
run,block,thread,state,uniques,visited,visited_percent,vts,total_visited
0,,,,,178913,100.998,250,31481577
1,,,,,178913,100.998,500,31480442
2,,,,,178913,100.998,750,31480607
...

real    0m4.468s
user    0m4.230s
sys     0m0.176s
```

Full output data is available at [EXP-10-dining-philosophers-processes-1.csv](./data/EXP-10-dining-philosophers-processes-1.csv).

---

Then, we set `kProcesses = 12` and `kStateSpaceSize = 531440`.

```
$ time ./build/grapple -s 1736331306 -n 100 -q
run,block,thread,state,uniques,visited,visited_percent,vts,total_visited
0,,,,,536054,100.868,250,51628961
1,,,,,536585,100.968,500,51627548
2,,,,,536585,100.968,750,51624312
...

real    1m7.006s
user    1m6.729s
sys     0m0.212s
```

Full output data is available at [EXP-10-dining-philosophers-processes-2.csv](./data/EXP-10-dining-philosophers-processes-2.csv).

---

For the last experiment, we set `kProcesses = 13` and `kStateSpaceSize = 1594322`.

```
$ time ./build/grapple -s 1736331306 -n 300 -q

real    3m41.322s
user    3m40.929s
sys     0m0.276s
```

Full output data is available at [EXP-10-dining-philosophers-processes-3.csv](./data/EXP-10-dining-philosophers-processes-3.csv).

## Evaluation

| Model | Hash Table Utilization | # of VTs to explore |
|-------|------------------------|---------------------|
| DP-11 | ~68% | 100% in 250 |
| DP-12 | ~203% | 100% in 250 |
| DP-13 | ~608% | 100% in 53250 |

## Conclusion, Discussion

The state space size of DP-13 is 9x the state space size of DP-11. However, exploring about 100% of DP-13's state space takes 213x as many VTs as for DP-11.

Thus, our hypothesis is confirmed on the dining philosophers model.